In [1]:
# imports
import numpy as np
import os
import tensorflow as tf
import pathlib

from keras import applications
from tensorflow import keras
from tensorflow.keras.models import Sequential

In [2]:
path = "data"
batch_size = 32
IMG_SIZE = 224
epochs = 150

In [3]:
def generate_model():
    load_data(path)
    class_names, train_ds, val_ds = train_val_split(path)
    base_model = define_model(len(class_names))
    train_model(epochs, base_model, train_ds, val_ds)
    save_model(base_model)

### 0. Load images from data folder

In [4]:
def load_data(path):
    data_dir = pathlib.Path(path)
    image_count = len(list(data_dir.glob('*/*')))
    print(f"Successfully loaded {image_count} images")
    return data_dir

### 1. Split data into train and validation dataset

In [5]:
# 20% of images used for validation
def train_val_split(data_dir):
    train_ds = tf.keras.preprocessing.image_dataset_from_directory(
        data_dir,
        labels = 'inferred',
        validation_split = 0.2,
        subset = "training",
        seed = 123,
        image_size = (IMG_SIZE, IMG_SIZE),
        batch_size = batch_size)

    val_ds = tf.keras.utils.image_dataset_from_directory(
        data_dir,
        labels = 'inferred',
        validation_split = 0.2,
        subset = "validation",
        seed = 123,
        image_size = (IMG_SIZE, IMG_SIZE),
        batch_size = batch_size
    )
    
    return train_ds.class_names, train_ds, val_ds

### 2. Define model

In [6]:
# after multiple researches and test, ResNet 50 seems to be the best convolutional neural network

from tensorflow.keras.applications import ResNet50
from tensorflow.python.keras.layers import Dense, Flatten, GlobalAveragePooling2D

def define_model(num_classes):
    IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)

    # Create the base model from the pre-trained model MobileNet V2
    base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

    data_augmentation = keras.Sequential(
      [
        tf.keras.layers.RandomFlip("horizontal",
                          input_shape=(IMG_SIZE,
                                      IMG_SIZE,
                                      3)),
        tf.keras.layers.RandomRotation(0.1),
        tf.keras.layers.RandomZoom(0.1),
      ]
    )
    
    base_model.trainable = False
    
    model = tf.keras.Sequential([
      data_augmentation,
      base_model,
      tf.keras.layers.GlobalAveragePooling2D(),
      tf.keras.layers.Dense(521, activation='relu'),
      tf.keras.layers.Dropout(0.3),
      tf.keras.layers.Dense(256, activation="relu"),
      tf.keras.layers.Dropout(0.15),
      tf.keras.layers.Dense(8)
    ])
    model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001), loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics = ['acc'])
    #model.summary()
    return model


### 3. Train Model

In [7]:
def train_model(epochs, base_model, train_ds, val_ds):
    history = base_model.fit(
      train_ds,
      validation_data = val_ds,
      epochs = epochs
    )

### 5. Save model

In [8]:
def save_model(model):
    model.save("model/model.h5")
    
    # quantization used to allow a lighter model for mobile architectures
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    
    tflite_model_quant = converter.convert()
    # Save the model.
    with open('model/model_quantized.tflite', 'wb') as f:
      f.write(tflite_model)

### 6. External Tests

In [9]:
from PIL import Image, ImageOps
def some_test(model):
    labels = ["Yeezy Slide", "Blazer", "Jordan 11", "Air Max 1", "Yeezy 700", "Yeezy 350", "Jordan 4","Jordan 1 High"]
    for test in os.listdir("test_images"):
        try:
            data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
            image = Image.open('test_images/' + test)
            size = (224, 224)
            image = ImageOps.fit(image, size, Image.ANTIALIAS)
            image_array = np.asarray(image)
            normalized_image_array = (image_array.astype(np.float32) / 127.0) - 1
            data[0] = normalized_image_array
            prediction = model.predict(data)
            print(test)
            print(
                "This image most likely belongs to {} with a {:.2f} percent confidence."
                .format(labels[np.argmax(prediction)], 100 * np.max(prediction))
            )
        except:
            pass #got errors for .DS files

In [10]:
##main
if os.path.isfile('model/model_quantized.tflite') or os.path.isfile('model/model.h5'):
    print("Model already trained, loading weights..")
    model = tf.keras.models.load_model('model/model.h5', compile = False)
    print("Model successfully loaded!")
    print("Doing tests on test_images folder")
    some_test(model)
else:
    print("No model available, train a new one..")
    generate_model()

No model available, train a new one..
Successfully loaded 4813 images
Found 4813 files belonging to 8 classes.
Using 3851 files for training.
Found 4813 files belonging to 8 classes.
Using 962 files for validation.
Epoch 1/150
121/121 [==============================] - 129s 1s/step - loss: 1.9282 - acc: 0.2610 - val_loss: 1.7106 - val_acc: 0.3721
Epoch 2/150
121/121 [==============================] - 127s 1s/step - loss: 1.7048 - acc: 0.3677 - val_loss: 1.5876 - val_acc: 0.3992
Epoch 3/150
121/121 [==============================] - 144s 1s/step - loss: 1.6162 - acc: 0.4069 - val_loss: 1.4881 - val_acc: 0.4574
Epoch 4/150
  3/121 [..............................] - ETA: 1:54 - loss: 1.5222 - acc: 0.4479

KeyboardInterrupt: 